In [1]:
!pip3 install newsapi-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import spacy
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
api_key = '946472f8ad574082a31c9d0fcaf1e89d'
newsapi = NewsApiClient(api_key=api_key)

In [4]:
def crawl_news(query):
    all_results = []
    for pag in tqdm(range(1, 6)):
        pag_articles = newsapi.get_everything(q=query, sort_by='relevancy', page=pag)['articles']
        if len(pag_articles) == 0:
            break
        all_results.extend(pag_articles)
    return all_results

In [5]:
tesla_news = crawl_news('tesla')

100%|██████████| 5/5 [00:01<00:00,  3.37it/s]


In [8]:
df = pd.read_csv('BBC news dataset.csv', usecols=range(1, 3))

In [9]:
df.drop_duplicates('description', inplace=True)

In [10]:
descriptions = df['description'].values

In [11]:
def pre_process_text(text):

    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
processed_descriptions = []
for description in tqdm(descriptions):
    processed_descriptions.append(' '.join(pre_process_text(description)))

100%|██████████| 2128/2128 [00:04<00:00, 522.54it/s]


In [14]:
nlp = spacy.load('en_core_web_sm')

sent_vecs = {}
docs = []

for index, description in enumerate(tqdm(processed_descriptions)):
    doc = nlp(description)
    docs.append(doc)
    sent_vecs[index] = doc.vector

100%|██████████| 2128/2128 [01:39<00:00, 21.40it/s]


In [15]:
vectors = list(sent_vecs.values())

In [16]:
vectors = np.array(vectors)